In [1]:
!pip install transformers[torch]
!pip install datasets
import os
import pandas as pd
import torch
import transformers as tr
from datasets import Dataset, load_metric
import numpy as np
from transformers import Trainer
from sklearn.metrics import classification_report, accuracy_score

import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv("https://github.com/AnasElbattra/News-Understanding-NLP/raw/main/TOPICS/DATA/MN-DS-news-classification.csv")
df

,data_id,id,date,source,title,content,author,url,published,published_utc,collection_utc,category_level_1,category_level_2
0,1809,abcnews--2019-10-31--Virginia mom charged with...,2019-10-31,abcnews,Virginia mom charged with murder in 2-year-old...,The Virginia woman whose 2-year-old son was fo...,NaN,https://abcnews.go.com/US/wireStory/virginia-m...,"Thu, 31 Oct 2019 16:49:56 -0400",1572554996,1572559512,"crime, law and justice",crime
1,1980,abcnews--2019-11-07--2 escaped murder suspects...,2019-11-07,abcnews,2 escaped murder suspects arrested at US-Mexic...,Authorities are trying to determine if anyone ...,NaN,https://abcnews.go.com/US/wireStory/escaped-mu...,"Thu, 07 Nov 2019 00:13:12 -0500",1573103592,1573131986,"crime, law and justice",crime
2,1995,abcnews--2019-11-07--Family turns in escaped b...,2019-11-07,abcnews,"Family turns in escaped boy, 13, suspected in ...",A 13-year-old suspect in a double homicide who...,NaN,https://abcnews.go.com/US/wireStory/family-tur...,"Thu, 07 Nov 2019 07:39:54 -0500",1573130394,1573131982,"crime, law and justice",crime
3,2740,abcnews--2019-12-02--Mother charged with murde...,2019-12-02,abcnews,Mother charged with murder in deaths of 2 youn...,The mother of two young children found hanging...,NaN,https://abcnews.go.com/US/wireStory/mother-cha...,"Mon, 02 Dec 2019 11:30:59 -0500",1575304259,1575308811,"crime, law and justice",crime
4,7038,ageofautism--2019-04-12--Physician Father and ...,2019-04-12,ageofautism,"Physician, Father and Caretaker of 29 Year Old...","""One family member said Derek “can be violent ...",Age of Autism,http://feedproxy.google.com/~r/ageofautism/~3/...,2019-04-12 09:00:00+00:00,1555074000,1567543083,"crime, law and justice",crime
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10912,907640,therussophileorg--2019-12-15--Iran to build ov...,2019-12-15,therussophileorg,"Iran to build over 30,000 housing units in Syr...",This post was originally published on this sit...,News Desk,https://www.therussophile.org/iran-to-build-ov...,"Sun, 15 Dec 2019 10:42:50 +0000",1576424570,1576413702,"conflict, war and peace",post-war reconstruction
10913,892720,therussophileorg--2019-10-12--Ukraine has no m...,2019-10-12,therussophileorg,Ukraine has no money for reconstruction of Don...,This post was originally published on this sit...,Michael Sullivan,https://www.therussophile.org/ukraine-has-no-m...,"Sat, 12 Oct 2019 20:56:01 +0000",1570928161,1570922006,"conflict, war and peace",post-war reconstruction
10914,870499,therussophileorg--2019-07-06--Irans contributi...,2019-07-06,therussophileorg,Iran’s contribution is needed to rebuild Iraq ...,This\n\n[post](http://www.presstv.ir/Detail/20...,Michael Sullivan,https://www.therussophile.org/irans-contributi...,2019-07-06 17:51:38+00:00,1562449898,1567536625,"conflict, war and peace",post-war reconstruction
10915,887334,therussophileorg--2019-09-25--Iraqi president ...,2019-09-25,therussophileorg,Iraqi president demands international help for...,This\n\n[post](http://www.presstv.ir/Detail/20...,Michael Sullivan,https://www.therussophile.org/iraqi-president-...,2019-09-25 16:58:01+00:00,1569445081,1570222223,"conflict, war and peace",post-war reconstruction


In [3]:
df = df[['title', 'content', 'category_level_1', 'category_level_2']]
df.rename(columns={'category_level_1': 'label_1', 'category_level_2': 'label_2'}, inplace=True)
df

,title,content,label_1,label_2
0,Virginia mom charged with murder in 2-year-old...,The Virginia woman whose 2-year-old son was fo...,"crime, law and justice",crime
1,2 escaped murder suspects arrested at US-Mexic...,Authorities are trying to determine if anyone ...,"crime, law and justice",crime
2,"Family turns in escaped boy, 13, suspected in ...",A 13-year-old suspect in a double homicide who...,"crime, law and justice",crime
3,Mother charged with murder in deaths of 2 youn...,The mother of two young children found hanging...,"crime, law and justice",crime
4,"Physician, Father and Caretaker of 29 Year Old...","""One family member said Derek “can be violent ...","crime, law and justice",crime
...,...,...,...,...
10912,"Iran to build over 30,000 housing units in Syr...",This post was originally published on this sit...,"conflict, war and peace",post-war reconstruction
10913,Ukraine has no money for reconstruction of Don...,This post was originally published on this sit...,"conflict, war and peace",post-war reconstruction
10914,Iran’s contribution is needed to rebuild Iraq ...,This\n\n[post](http://www.presstv.ir/Detail/20...,"conflict, war and peace",post-war reconstruction
10915,Iraqi president demands international help for...,This\n\n[post](http://www.presstv.ir/Detail/20...,"conflict, war and peace",post-war reconstruction


In [4]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, train_size=2000, random_state=42, stratify=df['label_1'])
df_train

,title,content,label_1,label_2
6583,Hungary&apos;s far-right government passes law...,Hungary‘s far-right government has passed a la...,science and technology,scientific research
8199,"American Medical Association, which pushes ant...",(Natural News) To the brainiacs at the America...,science and technology,scientific institution
3619,Bournemouth fan who threw pie onto pitch durin...,A football fan faces a future ban after he was...,sport,disciplinary action in sport
9002,Mariah Carey &apos;freaks out&apos; as she mar...,"Mariah Carey has said she is ""freaking out"" ov...",human interest,anniversary
3313,Best London gym packages 2019,The New Year is all about setting resolutions ...,sport,sport industry
...,...,...,...,...
8062,Presidential Misconduct: Some Historical Persp...,If you think Trump’s behavior is the worst in ...,politics,political dissent
9481,EW. Professional Cyclist Shares Shocking Pict...,EW. Professional Cyclist Shares Shocking Pictu...,sport,bodybuilding
1203,North Yorkshire flooding: Roads destroyed and ...,Flash flooding has wiped out roads and destroy...,"disaster, accident and emergency incident",disaster
1136,Horror moment killer neighbour climbs window b...,This was the horrifying moment a man climbed f...,"crime, law and justice",crime


In [5]:
# label_1 = df_train["label_1"].unique().tolist()
# label_1_dict = {label_1[i]: i for i in range(len(label_1))}
# df_train["label_1"] = df_train["label_1"].map(label_1_dict)
# df_test["label_1"] = df_test["label_1"].map(label_1_dict)

In [6]:
df_train

,title,content,label_1,label_2
6583,Hungary&apos;s far-right government passes law...,Hungary‘s far-right government has passed a la...,science and technology,scientific research
8199,"American Medical Association, which pushes ant...",(Natural News) To the brainiacs at the America...,science and technology,scientific institution
3619,Bournemouth fan who threw pie onto pitch durin...,A football fan faces a future ban after he was...,sport,disciplinary action in sport
9002,Mariah Carey &apos;freaks out&apos; as she mar...,"Mariah Carey has said she is ""freaking out"" ov...",human interest,anniversary
3313,Best London gym packages 2019,The New Year is all about setting resolutions ...,sport,sport industry
...,...,...,...,...
8062,Presidential Misconduct: Some Historical Persp...,If you think Trump’s behavior is the worst in ...,politics,political dissent
9481,EW. Professional Cyclist Shares Shocking Pict...,EW. Professional Cyclist Shares Shocking Pictu...,sport,bodybuilding
1203,North Yorkshire flooding: Roads destroyed and ...,Flash flooding has wiped out roads and destroy...,"disaster, accident and emergency incident",disaster
1136,Horror moment killer neighbour climbs window b...,This was the horrifying moment a man climbed f...,"crime, law and justice",crime


In [7]:
dataset = Dataset.from_pandas(df_train)
dataset = dataset.train_test_split(test_size=0.2)
dataset["train"]["label_1"]

['labour',
 'politics',
 'weather',
 'society',
 'religion and belief',
 'sport',
 'human interest',
 'health',
 'weather',
 'religion and belief',
 'education',
 'society',
 'arts, culture, entertainment and media',
 'environment',
 'weather',
 'labour',
 'education',
 'society',
 'human interest',
 'religion and belief',
 'politics',
 'economy, business and finance',
 'science and technology',
 'environment',
 'crime, law and justice',
 'society',
 'conflict, war and peace',
 'religion and belief',
 'society',
 'labour',
 'health',
 'conflict, war and peace',
 'sport',
 'education',
 'conflict, war and peace',
 'sport',
 'labour',
 'human interest',
 'conflict, war and peace',
 'sport',
 'disaster, accident and emergency incident',
 'environment',
 'conflict, war and peace',
 'conflict, war and peace',
 'crime, law and justice',
 'labour',
 'education',
 'conflict, war and peace',
 'conflict, war and peace',
 'human interest',
 'disaster, accident and emergency incident',
 'health',


In [8]:
def to_tokens(tokenizer):
    def apply(x):
        token_res = tokenizer(x["content"], text_target=x["label_1"],
                              return_tensors="pt", truncation=True, padding=True)
        return {
            "input_ids": token_res["input_ids"].tolist(),
            "attention_mask": token_res["attention_mask"].tolist(),
            "decoder_input_ids": token_res["input_ids"].tolist(),  # Specify decoder_input_ids here
        }
    return apply


In [9]:
tokenizer = tr.AutoTokenizer.from_pretrained("t5-small")

In [10]:
# label_lookup = {0: "fake", 1: "real"}
to_tokens = to_tokens(tokenizer)
tokenized_dataset = dataset.map(to_tokens, batched=True, remove_columns=["content", "label_1"])

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
local_training_root = "./"
checkpoint_name = "test-trainer"
local_checkpoint_path = os.path.join(local_training_root, checkpoint_name)
training_args = tr.TrainingArguments(
    local_checkpoint_path,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=10,
    report_to=["tensorboard"],
)

In [12]:
t5_model = tr.AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [13]:
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = [tokenizer.decode(p, skip_special_tokens=True) for p in predictions[0]]
    labels = [tokenizer.decode(l, skip_special_tokens=True) for l in labels]
    return metric.compute(predictions=predictions, references=labels)


In [14]:
trainer = Trainer(
    model=t5_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB (GPU 0; 14.76 GiB total capacity; 13.72 GiB already allocated; 21.75 MiB free; 13.75 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model()
trainer.save_state()

In [ ]:
checkpoint_name = "test-trainer-deepspeed"
final_model_path = f"./llm04_fine_tuning/{checkpoint_name}"
trainer.save_model(output_dir=final_model_path)
fine_tuned_model = tr.AutoModelForSeq2SeqLM.from_pretrained(final_model_path)